In [1]:
# Set the path
import sys
sys.path.append("/home/ubuntu/ACT")

from act.cell_model import ACTCellModel
from act.simulator import ACTSimulator
from act.act_types import SimulationParameters, ConstantCurrentInjection, FilterParameters, ConductanceOptions, OptimizationParameters
from act.data_processing import *
from act.act_module import ACTModule, ACTModuleParameters

import numpy as np
import matplotlib.pyplot as plt

--No graphics will be displayed.


In [2]:
random_seed = 42
num_slices = 3
experiment_folder = f"output/LAA_segregated-{num_slices}/{random_seed}"
module_folder_1 = experiment_folder + "/module_1"
module_folder_2 = experiment_folder + "/module_2"

target_folder = module_folder_1 + "/target"

target_cell = ACTCellModel(
    path_to_hoc_file="/home/ubuntu/ACT/data/LAA/seg/template.hoc",
    path_to_mod_files="/home/ubuntu/ACT/data/LAA/seg/modfiles",
    cell_name="Cell_A_seg",
    passive=["ghdbar_hd","glbar_leak"],
    active_channels=["gbar_nap",
                     "gmbar_im", 
                     "gcabar_cadyn", 
                     "gsAHPbar_sAHP", 
                     "gkabar_kap",
                     "gbar_na3",
                     "gkdrbar_kdr"]
)

sim_par= SimulationParameters(
            h_v_init=-65.0,  # Consistent with nrngui settings
            h_tstop=1000,     # Simulation time in ms
            h_dt=0.1,      # Time step in ms
            h_celsius=6.3,   # Temperature in degrees Celsius
            CI=[ConstantCurrentInjection(amp=0.040,dur=800,delay=100, lto_hto=1),
                ConstantCurrentInjection(amp=0.045,dur=800,delay=100, lto_hto=1),
                ConstantCurrentInjection(amp=0.050,dur=800,delay=100, lto_hto=1),
                ConstantCurrentInjection(amp=0.055,dur=800,delay=100, lto_hto=1),
                ConstantCurrentInjection(amp=0.060,dur=800,delay=100, lto_hto=1),
                ConstantCurrentInjection(amp=0.0,dur=800,delay=100),
                ConstantCurrentInjection(amp=0.1,dur=800,delay=100),
                ConstantCurrentInjection(amp=0.2,dur=800,delay=100),
                ConstantCurrentInjection(amp=0.3,dur=800,delay=100),
                ConstantCurrentInjection(amp=0.4,dur=800,delay=100),
                ConstantCurrentInjection(amp=4.5,dur=800,delay=100, lto_hto=1),
                ConstantCurrentInjection(amp=5.0,dur=800,delay=100, lto_hto=1),
                ConstantCurrentInjection(amp=5.5,dur=800,delay=100, lto_hto=1),
                ConstantCurrentInjection(amp=6.0,dur=800,delay=100, lto_hto=1),
                ConstantCurrentInjection(amp=6.5,dur=800,delay=100, lto_hto=1)]
        )

In [3]:
# Run Target Cell on Current Injections
simulator = ACTSimulator(output_folder_name = ".")

# LTO
for i in range(len(sim_par.CI)):
    sim_params = SimulationParameters(
        sim_name = target_folder, 
        sim_idx = i, 
        h_v_init=-65,
        h_celsius = 6.3,
        h_dt = 0.1,
        h_tstop = 1000,
        CI = [sim_par.CI[i]])

    simulator.submit_job(target_cell, sim_params)


        ACTSimulator (2025)
        ----------
        When submitting multiple jobs, note that the cells must share modfiles.
        


In [4]:
simulator.run_jobs(len(sim_par.CI))

In [5]:
# Combine simulated traces into one dataset for convenience
combine_data(target_folder)

output/LAA_segregated-3/42/module_1/target


In [6]:
train_cell = ACTCellModel(
    path_to_hoc_file="/home/ubuntu/ACT/data/LAA/seg/template.hoc",
    path_to_mod_files="/home/ubuntu/ACT/data/LAA/seg/modfiles",
    cell_name="Cell_A_seg",
    passive=["ghdbar_hd","glbar_leak"],
    active_channels=["gbar_nap",
                     "gmbar_im", 
                     "gcabar_cadyn", 
                     "gsAHPbar_sAHP", 
                     "gkabar_kap",
                     "gbar_na3",
                     "gkdrbar_kdr"]
)

# Ion Channel Range Randomizer

In [7]:
random_state = np.random.RandomState(123)

#glbar_leak = 5.5e-5
#ghdbar_hd=2.3e-05
gmbar_im = 0.002
gbar_nap= 0.000142
gbar_na3=0.03
gkdrbar_kdr=0.0015
gcabar_cadyn = 6e-5
gsAHPbar_sAHP = 0.009
gkabar_kap = 0.000843

gbar_nap_range = (gbar_nap - random_state.uniform(0, gbar_nap / 2), gbar_nap + random_state.uniform(0, gbar_nap / 2))
gmbar_im_range = (gmbar_im - random_state.uniform(0, gmbar_im / 2), gmbar_im + random_state.uniform(0, gmbar_im / 2))
gcabar_cadyn_range = (gcabar_cadyn - random_state.uniform(0, gcabar_cadyn / 2), gcabar_cadyn + random_state.uniform(0, gcabar_cadyn / 2))
gsAHPbar_sAHP_range = (gsAHPbar_sAHP - random_state.uniform(0, gsAHPbar_sAHP / 2), gsAHPbar_sAHP + random_state.uniform(0, gsAHPbar_sAHP / 2))
gkabar_kap_range = (gkabar_kap - random_state.uniform(0, gkabar_kap / 2), gkabar_kap + random_state.uniform(0, gkabar_kap / 2))
gbar_na3_range = (gbar_na3 - random_state.uniform(0, gbar_na3 / 2), gbar_na3 + random_state.uniform(0, gbar_na3 / 2))
gkdrbar_kdr_range = (gkdrbar_kdr - random_state.uniform(0, gkdrbar_kdr / 2), gkdrbar_kdr + random_state.uniform(0, gkdrbar_kdr / 2))

# Module 1 (Bursting)

In [8]:
mod1 = ACTModule(
    ACTModuleParameters(
        module_folder_name=module_folder_1,
        cell= train_cell,
        target_traces_file = f"{target_folder}/combined_out.npy",
        sim_params= sim_par,
        optim_params= OptimizationParameters(
            conductance_options = [
                ConductanceOptions(variable_name="gbar_nap", low=gbar_nap_range[0], high= gbar_nap_range[1], n_slices=num_slices),
                ConductanceOptions(variable_name="gmbar_im", low=gmbar_im_range[0], high= gmbar_im_range[1],  n_slices=num_slices),
                ConductanceOptions(variable_name="gcabar_cadyn", low= gcabar_cadyn_range[0], high= gcabar_cadyn_range[1], n_slices=num_slices),
                ConductanceOptions(variable_name="gsAHPbar_sAHP", low= gsAHPbar_sAHP_range[0], high= gsAHPbar_sAHP_range[1], n_slices=num_slices),
                ConductanceOptions(variable_name="gkabar_kap", low= gkabar_kap_range[0], high= gkabar_kap_range[1], n_slices=num_slices),
                ConductanceOptions(variable_name="gbar_na3", low=gbar_na3_range[0], high= gbar_na3_range[1], n_slices=1), # average of range
                ConductanceOptions(variable_name="gbar_kdr", low=gkdrbar_kdr_range[0], high= gkdrbar_kdr_range[1], n_slices=1) # average of range 
            ],
            train_features=["i_trace_stats", "number_of_spikes", "spike_times", "spike_height_stats", "trough_times", "trough_height_stats", "lto-hto_amplitude", "lto-hto_frequency"],
            prediction_eval_method='features',
            save_file=f"{module_folder_1}/results/saved_metrics.json"
        )
    )
)

In [9]:
predicted_g_data_file = mod1.run()
print(train_cell.active_channels)

RUNNING THE MODULE
LOADING TARGET TRACES
SIMULATING TRAINING DATA

        ACTSimulator (2025)
        ----------
        When submitting multiple jobs, note that the cells must share modfiles.
        
/home/ubuntu/ACT/experiments/LA_A/not-filtered/output/LAA_segregated-3/42/module_1/train
TRAINING RANDOM FOREST REGRESSOR
Extracting features: ['I_mean', 'I_stdev', 'Num Spikes', 'Spike Time 1', 'Spike Time 2', 'Spike Time 3', 'Spike Time 4', 'Spike Time 5', 'Spike Time 6', 'Spike Time 7', 'Spike Time 8', 'Spike Time 9', 'Spike Time 10', 'Spike Time 11', 'Spike Time 12', 'Spike Time 13', 'Spike Time 14', 'Spike Time 15', 'Spike Time 16', 'Spike Time 17', 'Spike Time 18', 'Spike Time 19', 'Spike Time 20', 'Min Spike Height', 'Max Spike Height', 'Avg Spike Height', 'Std Spike Height', 'Trough Time 1', 'Trough Time 2', 'Trough Time 3', 'Trough Time 4', 'Trough Time 5', 'Trough Time 6', 'Trough Time 7', 'Trough Time 8', 'Trough Time 9', 'Trough Time 10', 'Trough Time 11', 'Trough Time 12', 

# Module 2 Spiking

In [11]:
bounds_var = 0.15
mod2 = ACTModule(
    ACTModuleParameters(
        module_folder_name=module_folder_2,
        cell= train_cell,
        target_traces_file = f"{target_folder}/combined_out.npy",
        sim_params= sim_par,
        optim_params= OptimizationParameters(
            conductance_options = [
                ConductanceOptions(variable_name="gbar_nap", prediction=train_cell.prediction["gbar_nap"], bounds_variation=train_cell.prediction["gbar_nap"] * bounds_var),
                ConductanceOptions(variable_name="gmbar_im", prediction=train_cell.prediction["gmbar_im"], bounds_variation=train_cell.prediction["gmbar_im"] * bounds_var),
                ConductanceOptions(variable_name="gcabar_cadyn", prediction=train_cell.prediction["gcabar_cadyn"], bounds_variation=train_cell.prediction["gcabar_cadyn"] * bounds_var),
                ConductanceOptions(variable_name="gsAHPbar_sAHP", prediction=train_cell.prediction["gsAHPbar_sAHP"], bounds_variation=train_cell.prediction["gsAHPbar_sAHP"] * bounds_var),
                ConductanceOptions(variable_name="gkabar_kap", prediction=train_cell.prediction["gkabar_kap"], bounds_variation=train_cell.prediction["gkabar_kap"] * bounds_var),
                ConductanceOptions(variable_name="gbar_na3", low=gbar_na3_range[0], high= gbar_na3_range[1], n_slices=num_slices), 
                ConductanceOptions(variable_name="gbar_kdr", low=gkdrbar_kdr_range[0], high= gkdrbar_kdr_range[1], n_slices=num_slices) 
            ],
            train_features=["i_trace_stats", "number_of_spikes", "spike_times", "spike_height_stats", "trough_times", "trough_height_stats", "lto-hto_amplitude", "lto-hto_frequency"],
            prediction_eval_method='features',
            save_file=f"{module_folder_1}/results/saved_metrics.json"
        )
    )
)

In [12]:
predicted_g_data_file = mod2.run()
print(train_cell.active_channels)

RUNNING THE MODULE
LOADING TARGET TRACES
SIMULATING TRAINING DATA

        ACTSimulator (2025)
        ----------
        When submitting multiple jobs, note that the cells must share modfiles.
        
/home/ubuntu/ACT/experiments/LA_A/not-filtered/output/LAA_segregated-3/42/module_2/train
TRAINING RANDOM FOREST REGRESSOR
Extracting features: ['I_mean', 'I_stdev', 'Num Spikes', 'Spike Time 1', 'Spike Time 2', 'Spike Time 3', 'Spike Time 4', 'Spike Time 5', 'Spike Time 6', 'Spike Time 7', 'Spike Time 8', 'Spike Time 9', 'Spike Time 10', 'Spike Time 11', 'Spike Time 12', 'Spike Time 13', 'Spike Time 14', 'Spike Time 15', 'Spike Time 16', 'Spike Time 17', 'Spike Time 18', 'Spike Time 19', 'Spike Time 20', 'Min Spike Height', 'Max Spike Height', 'Avg Spike Height', 'Std Spike Height', 'Trough Time 1', 'Trough Time 2', 'Trough Time 3', 'Trough Time 4', 'Trough Time 5', 'Trough Time 6', 'Trough Time 7', 'Trough Time 8', 'Trough Time 9', 'Trough Time 10', 'Trough Time 11', 'Trough Time 12', 

# Results & Metrics

In [13]:
from act.act_plot import *
plot_v_comparison(
    module_folder_2,
    predicted_g_data_file, 
    sim_par.CI,
    sim_par.h_dt
    )

plot_fi_comparison(
    module_folder_2, 
    sim_par.CI
    )

FileNotFoundError: [Errno 2] No such file or directory: 'output/LAA_segregated-3/42/module_2/target/combined_out.npy'

In [ ]:
from act.act_plot import *

mean, stdev = save_interspike_interval_comparison(
    module_folder_2,
    predicted_g_data_file,
    sim_par.CI, 
    sim_par.h_dt,
    first_n_spikes=5,
    save_file=f"{module_folder_2}/results/saved_metrics.json"
)

'''
gbar_nap= 0.000142
gmbar_im = 0.002
gcabar_cadyn = 6e-5
gsAHPbar_sAHP = 0.009
gkabar_kap = 0.000843
gbar_na3=0.03
gkdrbar_kdr=0.0015
'''

actual_g={"gbar_nap": 0.000142, "gmbar_im": 0.002, "gcabar_cadyn": 6e-5, "gsAHPbar_sAHP": 0.009, "gkabar_kap": 0.000843, "gbar_na3":0.03, "gkdrbar_kdr":0.0015}

    
save_prediction_g_mae(
    actual_g=actual_g,
    save_file=f"{module_folder_2}/results/saved_metrics.json"
)

save_feature_mae(
    module_folder_2,
    predicted_g_data_file,
    ["i_trace_stats", "number_of_spikes", "spike_times", "spike_height_stats", "trough_times", "trough_height_stats", "lto-hto_amplitude", "lto-hto_frequency"],
    sim_par.h_dt,
    first_n_spikes=5,
    CI_settings=sim_par.CI,
    save_file=f"{module_folder_2}/results/saved_metrics.json"
)


In [ ]:
from act.act_plot import *

g_names = ["gbar_nap",
            "gmbar_im", 
            "gcabar_cadyn", 
            "gsAHPbar_sAHP", 
            "gkabar_kap",
            "gbar_na3",
            "gkdrbar_kdr"]

for i in range(len(g_names)-1):
    plot_training_feature_mae_contour_plot(
        module_folder_2,
        sim_par.CI,
        sim_par.CI[0].delay,
        sim_par.h_dt,
        index1=0,
        index2=i+1,
        g_names=g_names,
        train_features=["i_trace_stats", "number_of_spikes", "spike_times", "spike_height_stats", "trough_times", "trough_height_stats", "lto-hto_amplitude", "lto-hto_frequency"],
        threshold=0,
        first_n_spikes=20,
        num_levels=100,
        results_filename=f"{module_folder_2}/results/Feature_MAE_Contour_Plot_{g_names[0]}_{g_names[i+1]}.png"
    )

In [ ]:
from act.act_plot import *

g_names = ["gbar_nap",
            "gmbar_im", 
            "gcabar_cadyn", 
            "gsAHPbar_sAHP", 
            "gkabar_kap",
            "gbar_na3",
            "gkdrbar_kdr"]

for i in range(len(g_names)-1):
    plot_training_fi_mae_contour_plot(
        module_folder_2,
        sim_par.CI, 
        sim_par.CI[0].delay,
        sim_par.h_dt,
        index1=0,
        index2=i+1,
        g_names=g_names,
        spike_threshold=0,
        results_filename=f"{module_folder_2}/results/FI_MAE_Contour_Plot_{g_names[0]}_{g_names[i+1]}.png"
    )

In [ ]:
from act import ACTPlot as actplt

g_names = ["gbar_nap",
            "gmbar_im", 
            "gcabar_cadyn", 
            "gsAHPbar_sAHP", 
            "gkabar_kap",
            "gbar_na3",
            "gkdrbar_kdr"]

for i in range(len(g_names)-1):
    plot_training_v_mae_contour_plot(
        module_folder_2,
        sim_par.CI, 
        sim_par.CI[0].delay,
        sim_par.h_dt,
        index1=0,
        index2=i+1,
        g_names=g_names,
        results_filename=f"{module_folder_2}/results/Voltage_MAE_Contour_Plot_{g_names[0]}_{g_names[i+1]}.png"
    )